In [1]:
from wrds2pg import get_process

In [2]:
tz = "America/New_York"

sas_code = f"""
    options nosource nonotes;
    options timezone='{tz}';
    
    data temp;
        set audit.feed34_revised_audit_opinions (obs=20 keep=audit_op_key file_accepted);
        temp_var = input(file_accepted, YMDDTTM.);
        drop file_accepted;
        rename temp_var=file_accepted;
    run;
    
    proc datasets lib=work;
        modify temp; 
        attrib file_accepted format=E8601LX.;
    run;

    proc export data=temp
        outfile=stdout 
        dbms=csv;
    run;
    
    /*
    proc contents data=temp;
    run;
    */
"""

In [3]:
p = get_process(sas_code, wrds_id="iangow")

In [4]:
p.readlines()

['AUDIT_OP_KEY,file_accepted\n',
 '66900,2001-03-26T00:00:00-05:00\n',
 '78252,2002-03-20T00:00:00-05:00\n',
 '3538,2003-03-27T14:55:13-05:00\n',
 '39225,2004-04-01T17:14:26-05:00\n',
 '84748,2005-03-31T10:34:42-05:00\n',
 '102979,2006-03-23T16:17:13-05:00\n',
 '120889,2007-03-09T16:12:21-05:00\n',
 '148621,2008-03-12T17:12:17-04:00\n',
 '171604,2009-03-13T16:47:14-04:00\n',
 '192814,2010-03-15T15:14:05-04:00\n',
 '156138,2000-08-24T00:00:00-04:00\n',
 '74991,2001-08-27T00:00:00-04:00\n',
 '45767,2002-08-26T00:00:00-04:00\n',
 '17785,2003-08-18T11:31:26-04:00\n',
 '51196,2004-07-22T12:57:20-04:00\n',
 '90725,2005-07-22T17:06:21-04:00\n',
 '109724,2006-07-17T13:42:14-04:00\n',
 '132153,2007-07-20T15:44:39-04:00\n',
 '159129,2008-07-11T17:20:22-04:00\n',
 '179725,2009-07-16T16:14:32-04:00\n']